## Amazon Rekognition

In following examples, I will show how by using boto3, we can interact with Rekognition service.

We need to start with creating Rekognition client:

In [24]:
import boto3

session = boto3.session.Session()
rekognition_client = session.client('rekognition')

### Detecting labels - Kitten example

In our first example, we will try to detect labels for the following picture:

![werter](./werter.jpeg)

To do that, we will use `detect_labels` method, which can accept image as a base64-encoded bytes or reference to image stored in S3 bucket. First, we will try to use our local image:

In [29]:
import base64
import pprint
    
response = rekognition_client.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': 'aws-ml-apis',
            'Name': 'rekognition/werter.jpeg'
        }
    }
)
pprint.pprint(response)

{'CelebrityFaces': [],
 'OrientationCorrection': 'ROTATE_0',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '79',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Mon, 12 Mar 2018 21:16:21 GMT',
                                      'x-amzn-requestid': '9bd3ae9d-263a-11e8-8187-2b691fd83f24'},
                      'HTTPStatusCode': 200,
                      'RequestId': '9bd3ae9d-263a-11e8-8187-2b691fd83f24',
                      'RetryAttempts': 0},
 'UnrecognizedFaces': []}
